In [1]:
from openai import OpenAI
import os
import re
import pandas as pd
from dotenv import load_dotenv
import requests
import base64
import subprocess
from IPython.display import display, Image
from PIL import Image as PILImage

In [2]:
import cv2
# import easyocr
import matplotlib.pyplot as plt
# import pytesseract
# import keras_ocr

In [3]:
# OpenAI
load_dotenv() #get the environment 
openai_API_KEY = os.getenv("OPENAI_API_KEY")
openai_client = OpenAI(api_key=openai_API_KEY)

In [4]:
path = os.path.dirname(os.getcwd()) # Parent directory
ouput_folder = path+'/results/predictions'

In [5]:
def get_outputs(output_path):   
    outputs = {}
    for output in os.listdir(output_path):
        # Check if the current item is a file and ends with '.txt'
        file_path = os.path.join(output_path, output)
        if os.path.isfile(file_path) and output.endswith('.txt'):
            filename = output.split('.')[0]
            # Open the individual file and read its content
            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read()
            outputs[filename] = text  # Store the filename and text in the dictionary
    return outputs

In [6]:
models = ['claude-3-5-sonnet-20240620', 'EasyOCR', 'gpt-4o', 'KerasOCR', 'Pytesseract', 'trOCR']
prompts = ['one-example_prompt', 'two-example_prompt', 'zero-shot_complex-prompt', 'zero-shot_simple-prompt',
                'refine_complex-prompt']
outputs = {}
for prompt in prompts:
    outputs[prompt] = {}  # Initialize the dictionary for each prompt
    for model in models:
        output_path = f'{ouput_folder}/{prompt}/{model}'
        outputs[prompt][model] = get_outputs(output_path)

In [47]:
outputs.keys()

dict_keys(['one-example_prompt', 'two-example_prompt', 'zero-shot_complex-prompt', 'zero-shot_simple-prompt', 'refine_complex-prompt'])

In [25]:
def callPostProcessing(max_tokens=800, prompt_parameter = None):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai_API_KEY}"
    } 
    payload = {
        "model": "gpt-4o",
        "messages": [
        {
            "role": "user",
            "content": [
            {
                "type": "text",
                "text": f"""This is an output from you. Clean it such that we only have the table without any separators, no comment from you: {prompt_parameter}
                """
            }
            ]
        }
        ],
        "max_tokens": max_tokens,
        "temperature": 0
    }
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    try:
        return response.json()["choices"][0]["message"]["content"]
    except:
        print(response.json()["error"]["message"])

In [53]:
print(outputs[prompt][model]['transcription1'])

Here's a recreation of the content of the table in the image:

| N° | DATE DU DÉCÈS | DÉSIGNATION DES PERSONNES DÉCÉDÉES OU ABSENTES | DATE DE DÉCÈS ou de la disparition | NOMS, PRÉNOMS des héritiers ou légataires | DROITS DE SUCCESSION | DROIT DE MUTATION | NUMÉROS du sommier | DATE | RECETTE | OBSERVATIONS |
|---|---------------|------------------------------------------------|-----------------------------------|-------------------------------------------|----------------------|---------------------|---------------------|------|---------|---------------|
| | | NOM | PRÉNOMS | DOMICILE | | | ACTIF | PASSIF | RESTANT NET | VALEUR | de l'expiration du délai de rectification | de l'exigibilité des droits | DATE | N° | |
| 403 | quatre 9bre | Payot | Antoinette | Braine l'Alleud | 14 mai 1919 | Payot Marie & autres | 16975 | 2950 | 11740 | 2011/1921 | 16 Db 1919 | 14 mars 1928 | | | |
| 403² | d° | Paulus | Mélanie | Nivelles | 28 Janvier 1919 | Paulus François | 1971 | | 1971 | 191/1919 

In [54]:
# Immediately save the post-processed outputs

# for prompt in prompts:
#     for model in models:
#         for key in outputs[prompt][model].keys():
#             post = callPostProcessing(prompt_parameter=outputs[prompt][model][key])
#             output_dir = os.path.join(path, 'results', 'postprocessed', prompt, model)
#             os.makedirs(output_dir, exist_ok=True)
#             file_path = os.path.join(output_dir, f"{key}.txt")
#             with open(file_path, 'w', encoding='utf-8') as file:
#                 file.write(post)

# Time: 59m 4.8s

* trOCR's outputs didn't get postprocessed well. Stil separators exsit. 
